In [1]:
import numpy as np
import pandas as pd
from Bio.Seq import Seq
from Bio.SeqUtils import nt_search

### Read data

In [2]:
Data = pd.read_csv("Fasta_Motif_CLIP_Site.csv", keep_default_na=False)

In [3]:
Data

,chrom,m6AStart,m6AEnd,Label,name,strand,txStart,txEnd,cdsStart,cdsEnd,...,LastExonLength,Pre,PreSequence,LastExonSequence,m6APosition,m6ASequence,POS,Up,Mid,Down
0,chr8,95718647,95718648,1,Setd6,+,95715931,95719010,95715936,95718781,...,535,2544,GCAGCATGGCGGCCCCCGCCAAGCGCGCGCGGGTAAGTGGCGGGTC...,GTACTGTGCATGCCTGCTGAGGAGTTCAGAGACTATAAAGAGCGGG...,172,GAC,2716,GTACTGTGCATGCCTGCTGAGGAGTTCAGAGACTATAAAGAGCGGG...,ACGGCTTCTTCGAAACAGTGTTCTGTTGACCCTACAGACCTACACC...,TGAGCTGGCGCGAACAACAAGCCTTACAGGTCCGCTATGGTCAGAA...
1,chr15,80212453,80212454,1,Mgat3,+,80173720,80215519,80210973,80212590,...,4547,37252,gATGGGGCGAGGAGAGGCTGCGGCGGACGCCAGCATCTccccgccg...,GATGAAGATGAGACGCTACAAGCTCTTTCTCATGTTCTGTATGGCT...,1481,AAC,38733,GATGAAGATGAGACGCTACAAGCTCTTTCTCATGTTCTGTATGGCT...,TCCTGCGGACCCCAGTGAGCACATGTATGCTCCTAAATACCTGCTC...,GCACTGAAGAGGGTGGGCGCCGGAACCAGGGCTCAGATGGAAGGCC...
2,chr2,29180075,29180076,1,Setx,+,29124991,29182471,29126886,29180125,...,3072,54408,GCCACTCAGCGGTCAGGCCGGTTGTTCCCCGCTGCTGCACTTTCCA...,GAAAACCAGCATTGGTATGAACTGATTCAGGATGCTCAGAAGCGTG...,676,GAC,55084,GAAAACCAGCATTGGTATGAACTGATTCAGGATGCTCAGAAGCGTG...,GGGGAGGCATGGCTCTGTGACTCATCACGTGCTGAGAAGCACTGAC...,AGCACCTCGTGAGGAGACTGTTGTCCACAGGCCAGCTAGTGGTACC...
3,chr5,77281251,77281252,1,Rest,+,77265490,77286432,77267940,77282984,...,5730,15212,AGCGTCCTGTGCTGGAATGTGCGGCTCCCGCGAGCTCGCGGCGCAG...,GTGAGAAGCCATTTAAATGTGATCAGTGCAATTATGTGGCCTCTAA...,549,GAC,15761,GTGAGAAGCCATTTAAATGTGATCAGTGCAATTATGTGGCCTCTAA...,GAAGTCAGCGGTGGCGGCGGAGACTAAAGCAGCAGAGGTGAAACAC...,AAAGAAAGAAGGATGCTGAGGCCCATCCCTCCGAAGAGCCTGTGAA...
4,chr4,127247100,127247101,1,Smim12,+,127243783,127247809,127246907,127247186,...,907,3119,AGTgggcggggcctcgggcgcgccccgcgggccACTGGGAACCTCG...,ATGTCATGTGGCCTGTGCTTTGGACCGTGGTGCGAACCTATGCCCC...,198,GAC,3317,ATGTCATGTGGCCTGTGCTTTGGACCGTGGTGCGAACCTATGCCCC...,CATCCTGGAGCGCCGAGAGGACCGAAAGCTGGATGAGATGCTAGGC...,CAGTCCTGAACAGAAACCGTCCAGAGAAGAACTAATGGAGGGCCCA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,chr4,129647810,129647811,1,Kpna6,-,129643979,129672767,129647991,129672680,...,4200,24588,GTCCTGCAGGTCCGCCATATTGTCTGCCGAGGCTACCGCTGAAGCT...,GCTTGGATAAAATTGAGTTTCTCCAGAGCCACGAGAACCAGGAAAT...,368,GAC,24956,GCTTGGATAAAATTGAGTTTCTCCAGAGCCACGAGAACCAGGAAAT...,CTCGACCTGTTCATCCCCTTCCCTGGAGGGAGCAAGCGCCCTCTGG...,GGGTTTCTCTTCCTTACACTATTTTGGCTGCACATGTCTTTAACCC...
996,chr17,24223916,24223917,1,Ccnf,-,24223231,24251409,24223897,24251310,...,1115,27063,GGGGGTGGGGCCCGCGCCTGCGTGAGGCGCTCCGTGCACTTGCCCC...,TCACAGCTCCCAGCAGACTCCTTGATGTCACTGTGGTCTACCTGAA...,429,GAC,27492,TCACAGCTCCCAGCAGACTCCTTGATGTCACTGTGGTCTACCTGAA...,TCACCAGCAGCCTGTAAAGCGCCAAAACCTGTCTGTACACAGTGAC...,AATGTTTACTGAGGGAGGGCTGCTGTTTCTCTAGGACACTGCCTAT...
997,chr2,130591682,130591683,1,Ubox5,-,130590001,130630038,130591805,130602610,...,2013,38024,AAGCGGAAGGTGAGGGCCGCTGGCCGGAAGTGGGTTGTGGAGCGGT...,AGCATCCAAGGAGTGTTTCTGGCCCAGAGTGTGCCTCCTGCAAACA...,331,GAC,38355,AGCATCCAAGGAGTGTTTCTGGCCCAGAGTGTGCCTCCTGCAAACA...,AGTGCCTCTGTTCTCTCTTGGGCTTGAATACCTCATGGTGTGTTAT...,CATAGGGCTGGGCTGGGGAAGACATCCCTACTCTCCCTGGCCCACC...
998,chr2,126629860,126629861,1,Gabpb1,-,126627441,126675568,126630229,126658577,...,2941,45186,GGAGCTCGGCCGCCGTGTCCCTCCGCGCCGACGGGGACTTTTCGGT...,GAGAGAGAAGCGCTTCAGAAACAGCTGGATGAAGCCAACCGAGAGG...,521,GAC,45707,GAGAGAGAAGCGCTTCAGAAACAGCTGGATGAAGCCAACCGAGAGG...,GACTTAATTTCTCAGAAACAGAAGTTGTAAGCGTGGTTGTTGTCTC...,AGATTTCGCCCAGGAACTTTTGTACAGCTTTTAAGTTGTCTCAGGT...


In [4]:
Regression = pd.read_csv("/pod/2/ke-lab/LUOZ/m6AAI/Figure2/Mouse_Motif_Regression_LastExon.csv")

In [5]:
PreSequence = Data["PreSequence"].tolist()
Up = Data["Up"].tolist()
Mid = Data["Mid"].tolist()
Down = Data["Down"].tolist()

In [6]:
from itertools import product
comb = list(product(["A","T","C","G"],repeat=5))
FiverMer = []
for i in range(len(comb)):
    value = "".join(comb[i])
    FiverMer.append(value)

In [7]:
MOTIF = dict(zip(FiverMer, [0]*1024))

In [8]:
for i in range(len(Data)):
    
    Pre = PreSequence[i]
    Pre = Pre.upper()
    up = Up[i]
    up = up.upper()
    mid = Mid[i]
    mid = mid.upper()
    down = Down[i]
    down =  down.upper()

    for j in range(0,97,1):
        DNA = mid[j:(j+5)]
        MOTIF[DNA] = MOTIF[DNA] + 1

In [9]:
MOTIF = pd.DataFrame(MOTIF, index=[0])
MOTIF = MOTIF.T
MOTIF.rename(columns={0:"Motif"}, inplace = True)

In [10]:
MOTIF

,Motif
AAAAA,296
AAAAT,161
AAAAC,213
AAAAG,188
AAATA,104
...,...
GGGCG,38
GGGGA,155
GGGGT,76
GGGGC,94


In [11]:
df = pd.DataFrame({"Index":range(0, 97)})
for i in range(len(Data)):
    
    Pre = PreSequence[i]
    Pre = Pre.upper()
    up = Up[i]
    up = up.upper()
    mid = Mid[i]
    mid = mid.upper()
    down = Down[i]
    down =  down.upper()

    New = []
    for j in range(0, 97, 1):
        string = mid[j:j+5]
        New.append(string)
    New = np.array(New)
    New = pd.DataFrame({Data.loc[i,"name"]:New})
    df = pd.merge(df, New, left_index=True, right_index=True)

In [12]:
df

,Index,Setd6_x,Mgat3_x,Setx_x,Rest,Smim12,Brpf1,Ak2,Pxn,Ankrd13c,...,Lrig1,Zfp518b,2610008E11Rik,Gpatch8_y,Galnt6,Kpna6,Ccnf,Ubox5,Gabpb1,Slc1a4_y
0,0,ACGGC,TCCTG,GGGGA,GAAGT,CATCC,CTCAG,CTTGC,GCGGC,GGCAG,...,GTCAG,TGAGC,GTAGA,ACAGT,CTCCT,CTCGA,TCACC,AGTGC,GACTT,TCACG
1,1,CGGCT,CCTGC,GGGAG,AAGTC,ATCCT,TCAGG,TTGCT,CGGCC,GCAGA,...,TCAGT,GAGCT,TAGAA,CAGTT,TCCTA,TCGAC,CACCA,GTGCC,ACTTA,CACGT
2,2,GGCTT,CTGCG,GGAGG,AGTCA,TCCTG,CAGGG,TGCTC,GGCCC,CAGAG,...,CAGTC,AGCTC,AGAAC,AGTTG,CCTAG,CGACC,ACCAG,TGCCT,CTTAA,ACGTT
3,3,GCTTC,TGCGG,GAGGC,GTCAG,CCTGG,AGGGG,GCTCT,GCCCT,AGAGC,...,AGTCA,GCTCT,GAACT,GTTGC,CTAGA,GACCT,CCAGC,GCCTC,TTAAT,CGTTG
4,4,CTTCT,GCGGA,AGGCA,TCAGC,CTGGA,GGGGA,CTCTA,CCCTT,GAGCT,...,GTCAG,CTCTG,AACTC,TTGCA,TAGAG,ACCTG,CAGCA,CCTCT,TAATT,GTTGT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,92,ATGCC,AGCCC,TTCTA,AGAAA,CCCCT,GGAGA,ACAAG,GGGCT,TCATT,...,TGTAC,GCTCA,CATAG,CTGGA,GCAAA,GGGAC,ACTAT,TAATA,TCTGC,ATGCT
93,93,TGCCA,GCCCA,TCTAT,GAAAA,CCCTA,GAGAG,CAAGG,GGCTC,CATTC,...,GTACA,CTCAG,ATAGT,TGGAA,CAAAC,GGACG,CTATG,AATAA,CTGCA,TGCTC
94,94,GCCAC,CCCAA,CTATA,AAAAA,CCTAA,AGAGG,AAGGC,GCTCT,ATTCA,...,TACAT,TCAGT,TAGTG,GGAAG,AAACC,GACGT,TATGG,ATAAA,TGCAA,GCTCC
95,95,CCACG,CCAAG,TATAG,AAAAC,CTAAA,GAGGA,AGGCC,CTCTT,TTCAT,...,ACATA,CAGTT,AGTGG,GAAGC,AACCC,ACGTG,ATGGT,TAAAG,GCAAA,CTCCA


#### Enhancer

In [13]:
from itertools import product
comb = list(product(["A","T","C","G"],repeat=5))
FiverMer = []
for i in range(len(comb)):
    value = "".join(comb[i])
    FiverMer.append(value)
Motif = dict(zip(FiverMer, [0]*1024))

In [14]:
EnhancerMotif = Regression.head(100)
List = EnhancerMotif["Name"].tolist()

In [15]:
for i in range(len(List)):
    motif = List[i]
    Motif[motif] = 1

In [16]:
Position = df
Position = Position.drop(["Index"], axis=1)
Position.replace(Motif, inplace=True)
Position["Enhancer"] =  Position.apply(lambda x:x.sum(),axis =1)
Enhancer = Position[["Enhancer"]]
Enhancer

,Enhancer
0,123
1,121
2,115
3,134
4,142
...,...
92,128
93,136
94,135
95,129


In [17]:
Enhancer.to_csv("mm10_Enhancer_CLIP.csv")

#### Silencer

In [18]:
from itertools import product
comb = list(product(["A","T","C","G"],repeat=5))
FiverMer = []
for i in range(len(comb)):
    value = "".join(comb[i])
    FiverMer.append(value)
Motif = dict(zip(FiverMer, [0]*1024))

In [19]:
SilencerMotif = Regression.tail(100)
List = SilencerMotif["Name"].tolist()

In [20]:
for i in range(len(List)):
    motif = List[i]
    Motif[motif] = 1

In [21]:
Position = df
Position = Position.drop(["Index"], axis=1)
Position.replace(Motif, inplace=True)
Position["Silencer"] =  Position.apply(lambda x:x.sum(),axis =1)
Silencer = Position[["Silencer"]]
Silencer

,Silencer
0,82
1,97
2,88
3,102
4,104
...,...
92,81
93,85
94,81
95,94


In [22]:
Silencer.to_csv("mm10_Silencer_CLIP.csv")

In [23]:
Data = pd.read_csv("Fasta_Motif_NonCLIP_Site.csv", keep_default_na=False)

In [24]:
Data

,chrom,m6AStart,m6AEnd,Label,name,strand,txStart,txEnd,cdsStart,cdsEnd,...,LastExonLength,Pre,PreSequence,LastExonSequence,m6APosition,m6ASequence,POS,Up,Mid,Down
0,chr1,43747600,43747601,0,Uxs1,-,43746965,43827800,43750230,43827708,...,3397,77438,GGGGCTGGTGCACGGGGGCGGGGCTCGGGCCGCCGCGCATTGTGCT...,GTTCCATTGGAGGAAGGATTGAACAAAGCCATCCACTACTTCCGGA...,2761,GAC,80199,GTTCCATTGGAGGAAGGATTGAACAAAGCCATCCACTACTTCCGGA...,GCTTCATTGCATTATCTTTGATTATGACCCTGGTGGTCCTCAGTCA...,TAAGACTGTAGGGAGTTTGGAGGTTCTTTAGAGATTCTTCAGAAAC...
1,chr10,122995463,122995464,0,Mon2,-,122992060,123076505,122995669,123075962,...,3773,80672,GGAGCTCGTTCTGGGCTTCTCGCTCGGCCCCGCCCCTCCAGAGACT...,TTGATGGCAATACCTGGTCACAAGTGATTGCCTTGTATCCGACGTT...,369,GAC,81041,TTGATGGCAATACCTGGTCACAAGTGATTGCCTTGTATCCGACGTT...,GCCTCCATGTTAACCAGGGCTCCTGGAGGAGCTGAGCCTTTGCCTG...,CAGTTGGTGACTTAAAAGTGAGCTTATGTACAGTTTGTTGTGTATG...
2,chr5,28458515,28458516,0,Shh,-,28456814,28467256,28457854,28466763,...,1789,8653,ACTGGAAGCCTTCAGGTGAAAATAAAGCCACAGCAGCCAGAGGGGG...,AGAACTCCGTGGCGGCCAAATCCGGCGGCTGTTTCCCGGGATCCGC...,87,GAC,8740,AGAACTCCGTGGCGGCCAAATCCGGCGGCTGTTTCCC,GGGATCCGCCACCGTGCACCTGGAGCAGGGCGGCACCAAGCTGGTG...,TGTACAGCGACTTCCTCACCTTCCTGGACCGCGACGAAGGCGCCAA...
3,chr19,38073767,38073768,0,Cep55,+,38055024,38074425,38057800,38073498,...,1128,18273,GGTGTCCTGGTTGATAAGTGGTCTGCGGCGGCGACGCGAGCGCTGT...,AAGCAGCTACATGGGTTTACCATCACAGAGCAGCCATTCCCACTCC...,470,GAC,18743,AAGCAGCTACATGGGTTTACCATCACAGAGCAGCCATTCCCACTCC...,GTGGTGAGCAGTGGCTATCCAAACTTGTAGCACTTCACGCTGTTGT...,TTGGTTATGGCTTAAAGCTACTGAGCAGTAAGCAAACATTTTATGC...
4,chr11,23738811,23738812,0,Rel,-,23736846,23770970,23742264,23770689,...,6194,27930,CACTAGTGACCGGAGCGCGAAGATTCGGGGGTGGGAAGGTGTGAGC...,AATCAAACTTATTTTCTCATGGTACAGTTTTACCAGAAATGCCCAG...,4228,GAC,32158,AATCAAACTTATTTTCTCATGGTACAGTTTTACCAGAAATGCCCAG...,AAACTCAGGCAGAAAATCACAAGTTTAAGGCCTGCACAGTCTGCAG...,AAAGGGCGTCCTGAGCTATAGTTCAGTGATAGAGCACCTAGCATGT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,chr7,51670097,51670098,0,Slc17a6,+,51621829,51671125,51622634,51669360,...,2101,47195,GCCAAAAGCAGCAGCCAAGGTTCCTTAGCTTCCTCTGTCCGTGGTC...,TCCCGTGAAGAATGGCAGTATGTCTTCCTCATTGCTGCACTCGTCC...,1073,AAC,48268,TCCCGTGAAGAATGGCAGTATGTCTTCCTCATTGCTGCACTCGTCC...,CCCATTCACTACCTGACAGTATCAAACCCTTCACATTTCAATAAAA...,GAAATAGACTTCAATGTATTTTTTATTACAACTTTGTACTGGTTGT...
996,chr12,69744470,69744471,0,Atp5s,+,69724961,69744658,69737954,69743833,...,940,18757,GAAGGAAAAGACCGAGGGCGAAGGGCCGGCCAGGCGCAGGGACGCA...,GAACCTCAAGTATTTGTTCTTAAGTGATCTTCCTGGAGTAAAAGAT...,752,AAC,19509,GAACCTCAAGTATTTGTTCTTAAGTGATCTTCCTGGAGTAAAAGAT...,TTAAAACAGAACGCAATACTTATCTACACCCAACTAGTTTTGCTGG...,GTGTGTTTGTTAGATTCTAATTTTGCTATCTCTCAAATACCTTTGG...
997,chr4,134803102,134803103,0,Tmem57,-,134802758,134853345,134804361,134853117,...,1806,48781,ggctggtgcggggcgggcgggccccggAATTGTCATTTCTTTGTTT...,GGCAAATTCTTCAGAAAGACCAGGAAATCAAGGACCTAAAACAGAA...,1461,AAC,50242,GGCAAATTCTTCAGAAAGACCAGGAAATCAAGGACCTAAAACAGAA...,ATTTCCTGTTGATTGGCATGTGTATCTCATTTAAAGCAAAATTAGG...,ACAGATCTTATTTTGGAGTAAAACAAACTTCTAGCTTTAGCCAGGT...
998,chr5,36954928,36954929,0,Ppp2r2c,+,36868512,36955078,36868820,36952642,...,2728,83838,GCGCGCGTCCCCTCGCCGCCGCCACCGCCAGGCGCCCGGACGCAGC...,TGTCATCATGACAGGGGCCTACAACAACTTCTTCCGCATGTTCGAC...,2578,AAC,86416,TGTCATCATGACAGGGGCCTACAACAACTTCTTCCGCATGTTCGAC...,TTCTCTGAGGGTGACGCCCTTGGAGGGTGTTGGGTATCTACGTGCT...,TTTTTTCTTCGGTGTCTCACAAATTATCCAACTGTTGAATAAAATT...


In [25]:
PreSequence = Data["PreSequence"].tolist()
Up = Data["Up"].tolist()
Mid = Data["Mid"].tolist()
Down = Data["Down"].tolist()

In [26]:
from itertools import product
comb = list(product(["A","T","C","G"],repeat=5))
FiverMer = []
for i in range(len(comb)):
    value = "".join(comb[i])
    FiverMer.append(value)

In [27]:
MOTIF = dict(zip(FiverMer, [0]*1024))

In [28]:
for i in range(len(Data)):
    
    Pre = PreSequence[i]
    Pre = Pre.upper()
    up = Up[i]
    up = up.upper()
    mid = Mid[i]
    mid = mid.upper()
    down = Down[i]
    down =  down.upper()

    for j in range(0,97,1):
        DNA = mid[j:(j+5)]
        MOTIF[DNA] = MOTIF[DNA] + 1

In [29]:
MOTIF = pd.DataFrame(MOTIF, index=[0])
MOTIF = MOTIF.T
MOTIF.rename(columns={0:"Motif"}, inplace = True)

In [30]:
MOTIF

,Motif
AAAAA,365
AAAAT,225
AAAAC,196
AAAAG,176
AAATA,179
...,...
GGGCG,31
GGGGA,117
GGGGT,87
GGGGC,82


In [31]:
df = pd.DataFrame({"Index":range(0, 97)})
for i in range(len(Data)):
    
    Pre = PreSequence[i]
    Pre = Pre.upper()
    up = Up[i]
    up = up.upper()
    mid = Mid[i]
    mid = mid.upper()
    down = Down[i]
    down =  down.upper()

    New = []
    for j in range(0, 97, 1):
        string = mid[j:j+5]
        New.append(string)
    New = np.array(New)
    New = pd.DataFrame({Data.loc[i,"name"]:New})
    df = pd.merge(df, New, left_index=True, right_index=True)

In [32]:
from itertools import product
comb = list(product(["A","T","C","G"],repeat=5))
FiverMer = []
for i in range(len(comb)):
    value = "".join(comb[i])
    FiverMer.append(value)
Motif = dict(zip(FiverMer, [0]*1024))

In [33]:
EnhancerMotif = Regression.head(100)
List = EnhancerMotif["Name"].tolist()

In [34]:
for i in range(len(List)):
    motif = List[i]
    Motif[motif] = 1


In [35]:
Position = df
Position = Position.drop(["Index"], axis=1)
Position.replace(Motif, inplace=True)
Position["Enhancer"] =  Position.apply(lambda x:x.sum(),axis =1)
Enhancer = Position[["Enhancer"]]
Enhancer

/home/luoz/miniconda3/envs/m6AAI/lib/python3.8/site-packages/pandas/core/frame.py:4383: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,Enhancer
0,103
1,101
2,105
3,94
4,107
...,...
92,119
93,105
94,97
95,91


In [36]:
Enhancer.to_csv("mm10_Enhancer_NonCLIP.csv")

In [37]:
from itertools import product
comb = list(product(["A","T","C","G"],repeat=5))
FiverMer = []
for i in range(len(comb)):
    value = "".join(comb[i])
    FiverMer.append(value)
Motif = dict(zip(FiverMer, [0]*1024))

In [38]:
SilencerMotif = Regression.tail(100)
List = SilencerMotif["Name"].tolist()

In [39]:
for i in range(len(List)):
    motif = List[i]
    Motif[motif] = 1


In [40]:
Position = df
Position = Position.drop(["Index"], axis=1)
Position.replace(Motif, inplace=True)
Position["Silencer"] =  Position.apply(lambda x:x.sum(),axis =1)
Silencer = Position[["Silencer"]]
Silencer

/home/luoz/miniconda3/envs/m6AAI/lib/python3.8/site-packages/pandas/core/frame.py:4383: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,Silencer
0,100
1,108
2,107
3,110
4,103
...,...
92,95
93,107
94,108
95,104


In [41]:
Silencer.to_csv("mm10_Silencer_NonCLIP.csv")